In [1]:
#gru many to one  5days

import pandas as pd
import numpy as np
'''
data preprocessing
'''

#주가데이터 파일을 로드
csv_data = pd.read_csv("./data/005930_200325.csv")
csv_data

,date,start,high,low,end,volume,foreign,person,company,wondollar
0,2020.3.25,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,2020.3.24,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,2020.3.23,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,2020.3.20,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,2020.3.19,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...
592,2017.10.23,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,2017.10.20,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,2017.10.19,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,2017.10.18,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [2]:
#필요한 부분을 가져온다
stdata = csv_data[["start","high", "low","end","volume","foreign", "person","company","wondollar"]]
stdata.head()

#전일대비 가격 변화율을 추가한다
price_change = -np.diff(stdata["end"].to_numpy())
price = stdata["end"].to_numpy()
price = price[1:]
price_change_rate = (price_change/price)*100
price_change_rate=np.append(price_change_rate,[0])
stdata['price_change_rate'] = price_change_rate

#캔들 (종가,시가,고가,저가) 관련 정보를 추가한다
#시가-종가
start = stdata["start"].to_numpy()
end = stdata["end"].to_numpy()
candle_end_start = end-start
stdata['candle_end_start'] = candle_end_start

#고가-저가 (당일변동폭) 
high = stdata["high"].to_numpy()
low = stdata["low"].to_numpy()
candle_high_low = high-low
stdata['candle_high_low'] = candle_high_low


#상승or하락 표시 상승:1 보함:0 하락:-1
up_or_down = np.zeros(597)

price_change_rate = stdata["price_change_rate"].to_numpy()
for idx, val in enumerate(price_change_rate):
    if val>0:
        up_or_down[idx]=1
    if val<0:
        up_or_down[idx]=-1
stdata['up_or_down'] = up_or_down

stdata


,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [3]:
#사용할 feature를 선택한다

input_data = stdata
input_data

,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [4]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fitted = scaler.fit(input_data)
input_arr = scaler.transform(input_data)
input_arr.shape

(597, 13)

In [5]:
#정규화 안할경우
#input_arr = input_data.to_numpy()
#input_arr.shape

In [6]:
#1일 -> len_of_day일 단위의 데이터로 변환 
len_of_day =5
feature_count = input_arr.shape[1]

#0행은 버린다 (정답이없기떄문)
input_arr=input_arr[1:]


#596일의 데이터로 10일짜리 데이터를 생성하면 596-10+1=587 개의 데이터가 생성된다
input_data_count = input_arr.shape[0]-len_of_day+1



#596*6열     [[feature갯수 개], 
#             [feature갯수개], 
#             [feature갯수개],
#             [feature갯수개],
#              ... ]
#
# 
#              을 [    [ [feature갯수],[feature갯수],[feature갯수]..........[feature갯수] (feature갯수개짜리 n일) ],
#                [ (feature갯수짜리 n일개) ],
#                [ (13개짜리 n일개) ],
#               .
#               .
#               . 
#               582개 
#    
#            ]  ( 582  ,10(일수) , 6(feature갯수) )    형태로 만들어야 한다


#inputdata1 : 10일 짜리 데이터 생성 

inputdata1 = np.empty((input_data_count,len_of_day,feature_count), dtype=np.float64)

for i in range(input_data_count):
    for j in range(len_of_day):
        inputdata1[i][j]=input_arr[i+j]
        
inputdata1.shape 

(592, 5, 13)

In [7]:
#정답 : 상승(변화율>0) or 하락 
#상승일경우 1 하락일경우 0

up_or_down = np.zeros(596)

price_change = -np.diff(stdata["end"].to_numpy())
for idx, val in enumerate(price_change):
    if val>0:
        up_or_down[idx]=1

up_or_down.shape
#1., 1., 0., 1., 0., 0., 0., 0

(596,)

In [8]:
#pytorch cross entry loss 에서는 target(정답)을 one hot encode 허자 얺음
#from sklearn import preprocessing

#(하락,상승)으로 one hot encoding
#
#up_or_down=up_or_down.astype(np.int64)
#num = np.unique(up_or_down, axis=0)
#num = num.shape[0]

#answer1 = np.eye(num)[up_or_down]
#answer1 = answer1[:582]
#answer1.shape

In [9]:
up_or_down=up_or_down.astype(np.int)
up_or_down=up_or_down[:input_data_count]
up_or_down = up_or_down.reshape(-1,1)
up_or_down.shape

(592, 1)

In [10]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [11]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [12]:
#train : 500 test : 87
inputdata1_train = inputdata1[:550]
inputdata1_test = inputdata1[550:]

up_or_down_train = up_or_down[:550]
up_or_down_test = up_or_down[550:]

In [13]:
class TrainDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_train
    self.y_data = up_or_down_train

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [14]:
class TestDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_test
    self.y_data = up_or_down_test

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [15]:
train_dataset = TrainDataset()
test_dataset = TestDataset()
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [16]:
data = dataloader

In [17]:
# feature 갯수
input_size = feature_count

#output size 바로 결과 예측을 위해서 1로 설정 
hidden_dim = 2

batch_size = 1 

# 15일
sequence_length = len_of_day

# one layer rnn이다.
n_layers = 2

#inputs = (batch_size * seq_len * input_size ) 인 행렬\
#hidden = (num_layers * batch_size * hidden_size ) 인 행렬
#output = (batch,seq_len,hidden)

In [18]:
#상승 또는 하락 판별 모델
class Model(nn.Module):
    def __init__(self, input_size, hidden_dim,output_size ,n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        h = self.init_hidden(batch_size)
        #c0 = self.init_hidden(batch_size)
        

        # Passing in the input and hidden state into the model and obtaining outputs
        out, h = self.rnn(x, h)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out[:,-1,:]
        #out = self.fc(out)
        #out = out[:,-1,:].view(1,1,1)
        return out,
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.rand(self.n_layers, batch_size, self.hidden_dim)
        hidden = nn.init.xavier_normal_(hidden).to(device)
        return hidden

In [19]:
#모델 초기화
model = Model(input_size, hidden_dim, output_size=2, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
nb_epochs = 20
lr=0.0001

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [20]:
#학습
#for epoch in range(nb_epochs + 1):
#     for batch_idx, samples in enumerate(dataloader):
for epoch in range(nb_epochs + 1):
    print("epoch: ", epoch)
    for batch_idx, (input_data,label) in enumerate(dataloader):
        optimizer.zero_grad()
        input_data = input_data.float().to(device)
        label = label.view(-1).long().to(device)
        output, = model(input_data)
        #print(input_data.shape)
        #print(output.shape)
        #print(label.shape)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if(batch_idx%10==0):
            print("epoch: ",epoch, "  iter:",batch_idx,"/502")
            print("Loss: {:.4f}".format(loss.item()))
            print("")

epoch:  0
epoch:  0   iter: 0 /502
Loss: 0.9952

epoch:  0   iter: 10 /502
Loss: 0.5282

epoch:  0   iter: 20 /502
Loss: 0.9498

epoch:  0   iter: 30 /502
Loss: 0.3018

epoch:  0   iter: 40 /502
Loss: 1.2241

epoch:  0   iter: 50 /502
Loss: 0.3656

epoch:  0   iter: 60 /502
Loss: 1.1676

epoch:  0   iter: 70 /502
Loss: 0.2829

epoch:  0   iter: 80 /502
Loss: 0.3339

epoch:  0   iter: 90 /502
Loss: 0.2982

epoch:  0   iter: 100 /502
Loss: 1.0114

epoch:  0   iter: 110 /502
Loss: 0.3247

epoch:  0   iter: 120 /502
Loss: 0.5055

epoch:  0   iter: 130 /502
Loss: 0.3222

epoch:  0   iter: 140 /502
Loss: 0.2233

epoch:  0   iter: 150 /502
Loss: 0.5411

epoch:  0   iter: 160 /502
Loss: 0.2879

epoch:  0   iter: 170 /502
Loss: 0.3896

epoch:  0   iter: 180 /502
Loss: 0.9487

epoch:  0   iter: 190 /502
Loss: 0.5056

epoch:  0   iter: 200 /502
Loss: 0.5584

epoch:  0   iter: 210 /502
Loss: 0.3646

epoch:  0   iter: 220 /502
Loss: 0.7204

epoch:  0   iter: 230 /502
Loss: 1.1616

epoch:  0   iter:

epoch:  3   iter: 390 /502
Loss: 0.6124

epoch:  3   iter: 400 /502
Loss: 0.4228

epoch:  3   iter: 410 /502
Loss: 0.4800

epoch:  3   iter: 420 /502
Loss: 0.9766

epoch:  3   iter: 430 /502
Loss: 0.6290

epoch:  3   iter: 440 /502
Loss: 0.6145

epoch:  3   iter: 450 /502
Loss: 1.1274

epoch:  3   iter: 460 /502
Loss: 0.6999

epoch:  3   iter: 470 /502
Loss: 1.2990

epoch:  3   iter: 480 /502
Loss: 0.5782

epoch:  3   iter: 490 /502
Loss: 0.5391

epoch:  3   iter: 500 /502
Loss: 0.6947

epoch:  3   iter: 510 /502
Loss: 0.7691

epoch:  3   iter: 520 /502
Loss: 1.0851

epoch:  3   iter: 530 /502
Loss: 1.1826

epoch:  3   iter: 540 /502
Loss: 0.4026

epoch:  4
epoch:  4   iter: 0 /502
Loss: 0.7118

epoch:  4   iter: 10 /502
Loss: 0.7349

epoch:  4   iter: 20 /502
Loss: 0.7431

epoch:  4   iter: 30 /502
Loss: 0.4038

epoch:  4   iter: 40 /502
Loss: 1.3885

epoch:  4   iter: 50 /502
Loss: 0.6066

epoch:  4   iter: 60 /502
Loss: 0.5936

epoch:  4   iter: 70 /502
Loss: 0.7951

epoch:  4   ite

epoch:  7   iter: 190 /502
Loss: 0.8061

epoch:  7   iter: 200 /502
Loss: 0.6307

epoch:  7   iter: 210 /502
Loss: 0.7516

epoch:  7   iter: 220 /502
Loss: 0.4477

epoch:  7   iter: 230 /502
Loss: 0.6903

epoch:  7   iter: 240 /502
Loss: 0.6228

epoch:  7   iter: 250 /502
Loss: 0.8634

epoch:  7   iter: 260 /502
Loss: 0.6289

epoch:  7   iter: 270 /502
Loss: 0.9460

epoch:  7   iter: 280 /502
Loss: 0.9141

epoch:  7   iter: 290 /502
Loss: 0.6351

epoch:  7   iter: 300 /502
Loss: 1.1748

epoch:  7   iter: 310 /502
Loss: 0.5480

epoch:  7   iter: 320 /502
Loss: 0.4834

epoch:  7   iter: 330 /502
Loss: 0.8748

epoch:  7   iter: 340 /502
Loss: 0.9631

epoch:  7   iter: 350 /502
Loss: 0.4024

epoch:  7   iter: 360 /502
Loss: 0.6356

epoch:  7   iter: 370 /502
Loss: 0.6642

epoch:  7   iter: 380 /502
Loss: 0.8207

epoch:  7   iter: 390 /502
Loss: 0.5516

epoch:  7   iter: 400 /502
Loss: 1.0099

epoch:  7   iter: 410 /502
Loss: 0.8090

epoch:  7   iter: 420 /502
Loss: 0.6878

epoch:  7   iter

epoch:  11
epoch:  11   iter: 0 /502
Loss: 0.8707

epoch:  11   iter: 10 /502
Loss: 0.6175

epoch:  11   iter: 20 /502
Loss: 0.3488

epoch:  11   iter: 30 /502
Loss: 0.3670

epoch:  11   iter: 40 /502
Loss: 0.6272

epoch:  11   iter: 50 /502
Loss: 0.4417

epoch:  11   iter: 60 /502
Loss: 0.5817

epoch:  11   iter: 70 /502
Loss: 0.4838

epoch:  11   iter: 80 /502
Loss: 1.0712

epoch:  11   iter: 90 /502
Loss: 0.6750

epoch:  11   iter: 100 /502
Loss: 0.5854

epoch:  11   iter: 110 /502
Loss: 0.4047

epoch:  11   iter: 120 /502
Loss: 0.6107

epoch:  11   iter: 130 /502
Loss: 0.7126

epoch:  11   iter: 140 /502
Loss: 0.3924

epoch:  11   iter: 150 /502
Loss: 0.8174

epoch:  11   iter: 160 /502
Loss: 0.8624

epoch:  11   iter: 170 /502
Loss: 1.0367

epoch:  11   iter: 180 /502
Loss: 0.7818

epoch:  11   iter: 190 /502
Loss: 0.5284

epoch:  11   iter: 200 /502
Loss: 0.4302

epoch:  11   iter: 210 /502
Loss: 0.7477

epoch:  11   iter: 220 /502
Loss: 0.5093

epoch:  11   iter: 230 /502
Loss: 

epoch:  14   iter: 310 /502
Loss: 0.6782

epoch:  14   iter: 320 /502
Loss: 0.6428

epoch:  14   iter: 330 /502
Loss: 0.7816

epoch:  14   iter: 340 /502
Loss: 0.7408

epoch:  14   iter: 350 /502
Loss: 0.5187

epoch:  14   iter: 360 /502
Loss: 0.7667

epoch:  14   iter: 370 /502
Loss: 0.3872

epoch:  14   iter: 380 /502
Loss: 0.7524

epoch:  14   iter: 390 /502
Loss: 0.7296

epoch:  14   iter: 400 /502
Loss: 1.0621

epoch:  14   iter: 410 /502
Loss: 1.0912

epoch:  14   iter: 420 /502
Loss: 0.3816

epoch:  14   iter: 430 /502
Loss: 0.6323

epoch:  14   iter: 440 /502
Loss: 0.6718

epoch:  14   iter: 450 /502
Loss: 0.6172

epoch:  14   iter: 460 /502
Loss: 0.6350

epoch:  14   iter: 470 /502
Loss: 0.5229

epoch:  14   iter: 480 /502
Loss: 0.5385

epoch:  14   iter: 490 /502
Loss: 0.5564

epoch:  14   iter: 500 /502
Loss: 0.7524

epoch:  14   iter: 510 /502
Loss: 0.4322

epoch:  14   iter: 520 /502
Loss: 0.7676

epoch:  14   iter: 530 /502
Loss: 0.6180

epoch:  14   iter: 540 /502
Loss: 

epoch:  18   iter: 100 /502
Loss: 0.6075

epoch:  18   iter: 110 /502
Loss: 0.4959

epoch:  18   iter: 120 /502
Loss: 0.7998

epoch:  18   iter: 130 /502
Loss: 0.7009

epoch:  18   iter: 140 /502
Loss: 0.5940

epoch:  18   iter: 150 /502
Loss: 0.6862

epoch:  18   iter: 160 /502
Loss: 0.9313

epoch:  18   iter: 170 /502
Loss: 0.7337

epoch:  18   iter: 180 /502
Loss: 0.8070

epoch:  18   iter: 190 /502
Loss: 0.5705

epoch:  18   iter: 200 /502
Loss: 0.7419

epoch:  18   iter: 210 /502
Loss: 1.0254

epoch:  18   iter: 220 /502
Loss: 0.5888

epoch:  18   iter: 230 /502
Loss: 0.6649

epoch:  18   iter: 240 /502
Loss: 0.5088

epoch:  18   iter: 250 /502
Loss: 0.5495

epoch:  18   iter: 260 /502
Loss: 0.6150

epoch:  18   iter: 270 /502
Loss: 1.0524

epoch:  18   iter: 280 /502
Loss: 0.5858

epoch:  18   iter: 290 /502
Loss: 0.6338

epoch:  18   iter: 300 /502
Loss: 0.8297

epoch:  18   iter: 310 /502
Loss: 0.7733

epoch:  18   iter: 320 /502
Loss: 0.5802

epoch:  18   iter: 330 /502
Loss: 

In [20]:
#평가
torch.no_grad()


In [32]:
ip, ans = test_dataset.__getitem__(0)

In [33]:
ip = ip.float()
ip = ip.view(-1,10,6).to(device)
ip.shape
ip

tensor([[[0.5808, 0.5908, 0.3101, 1.0000, 0.2824, 0.3430],
         [0.6178, 0.6947, 0.2202, 1.0000, 0.1828, 0.3400],
         [0.2732, 0.4321, 0.1753, 0.0000, 0.1608, 0.3385],
         [0.2478, 0.4412, 0.1124, 0.0000, 0.1470, 0.3380],
         [0.3388, 0.4779, 0.1798, 0.0000, 0.1784, 0.3403],
         [0.3789, 0.3771, 0.1573, 0.5000, 0.0900, 0.3391],
         [0.3990, 0.5420, 0.0404, 1.0000, 0.0855, 0.3388],
         [0.4373, 0.5969, 0.1663, 1.0000, 0.1330, 0.3391],
         [0.3137, 0.4504, 0.0449, 0.0000, 0.0741, 0.3394],
         [0.3639, 0.5267, 0.0989, 0.0000, 0.1406, 0.3385]]], device='cuda:0')

In [45]:
output, = model(ip)

In [65]:
output.shape

tensor([[[-0.7094],
         [-0.8642],
         [-0.9461],
         [-0.9789],
         [-0.9914],
         [-0.9961],
         [-0.9981],
         [-0.9989],
         [-0.9993],
         [-0.9994]]], device='cuda:0', grad_fn=<CudnnRnnBackward>)

In [66]:
s = output[:,-1,:]
s

tensor([[-0.9994]], device='cuda:0', grad_fn=<SliceBackward>)

In [67]:
s = s.view(-1)
s.shape

torch.Size([1])

In [23]:
ind = torch.rand(1,10,1)

In [24]:
ind.shape

torch.Size([1, 10, 1])

In [86]:
a = ind[:,-1,:].view(1,1,1)

a.shape
a

NameError: name 'ind' is not defined

In [40]:
loss = nn.CrossEntropyLoss()
input = torch.randn(1,2, requires_grad=True)
input.shape

torch.Size([1, 2])

In [41]:
target = torch.empty(1, dtype=torch.long).random_(5)
target.shape

torch.Size([1])

In [42]:

output  = loss(input, target)
output

tensor(3.1876, grad_fn=<NllLossBackward>)

In [130]:
output.backward()

In [34]:
input = torch.randn(1,10,2, requires_grad=True)
input

tensor([[[-0.7794,  0.8997],
         [-1.1116,  1.4836],
         [-0.3597,  0.4893],
         [-0.8259,  1.5241],
         [ 1.8444, -0.0579],
         [-0.2825,  0.5332],
         [-0.2259,  0.8802],
         [-0.1652,  1.3544],
         [ 1.9278,  1.1573],
         [ 0.7429,  0.3330]]], requires_grad=True)

In [35]:
z = input[:,-1,:]
z

tensor([[0.7429, 0.3330]], grad_fn=<SliceBackward>)

In [22]:
gfg =input.view(-1,2)
gfg.shape

torch.Size([10, 2])

In [24]:
gfg[-1].shape

torch.Size([2])

In [50]:
gfg

tensor([[ 0.5674,  0.3924,  1.1496, -0.8034,  1.1769],
        [ 0.8873,  1.0448, -0.3050,  0.4956,  0.7239],
        [-1.0780, -1.1770,  0.5911,  1.1897, -1.9601],
        [-0.5423,  1.2289, -1.0378, -0.6973,  0.4268],
        [-2.3498,  2.0009,  0.1865,  0.9067, -1.5810],
        [ 0.5034,  0.9189, -0.2453,  0.3872, -0.1004],
        [-1.1046, -0.3391, -0.3407,  0.1195,  0.3654],
        [ 1.6103, -0.1358, -1.4826, -0.1507,  0.1352],
        [-0.0906, -0.5085,  0.3033,  1.0276,  1.2815],
        [-0.8237,  1.2572,  1.3831,  0.2518, -0.9058],
        [ 0.1965, -1.2193, -0.5231,  0.1756, -0.6451],
        [ 0.9360, -0.1354,  0.9838,  0.0504, -0.1134]], grad_fn=<ViewBackward>)